In [2]:
import sys
sys.path.append('C:\\projects\\UTNCE\\src\\utnce\\utnce202402')

In [3]:
from simplify import *
from prepare import *
from routing import *
from percolation_plot import *
from generate import *
from transfer import *
from copy import deepcopy
from shapely.geometry import Point
from pyproj import Proj, Transformer

In [4]:
osm_path = "C:\\projects\\UTNCE\\data\\Amsterdam.large.pbf"

# Subway-Amsterdam

## Generate subway networks

In [5]:
sub_stations, edges, nodes, sub_routes, check_name = prepare_metro(osm_path)

AttributeError: 'NoneType' object has no attribute 'ExecuteSQL'

In [ ]:
edges

In [ ]:
nodes

In [ ]:
replacement_dict = {
     'Amsterdam Centraal': 'Centraal Station',
}


sub_routes, check_name = recheck_metro_routes(replacement_dict, sub_stations, sub_routes)

In [ ]:
sub_routes

In [ ]:
sub_order_route_dict, sub_shortest_path_pairs, sub_shortest_path_edges, sub_edges = metro(sub_stations, edges, nodes, sub_routes, city='Amsterdam')

In [ ]:
sub_order_route_dict

In [ ]:
sub_edges

In [ ]:
sub_shortest_path_edges

## Add columns to nodes

In [ ]:
sub_order_route_dict = add_ref_to_orderroutes_or_shortestpath_dict(sub_order_route_dict, sub_routes)

In [ ]:
sub_order_route_dict

In [ ]:
sub_new_nodes = add_columns_to_nodes(sub_order_route_dict, nodes)

In [ ]:
sub_new_nodes

In [ ]:
# sub_new_nodes.to_excel('.\city_sub_new_nodes11.xlsx',header = True, index = False)

### retrive all transfer stations on each route

In [ ]:
sub_transfer_stations_df = all_transfer_stations_df(sub_new_nodes)

In [ ]:
sub_transfer_stations_df

In [ ]:
len(sub_transfer_stations_df)

## Add columns to edges

In [ ]:
sub_shortest_path_edges = add_ref_to_orderroutes_or_shortestpath_dict(sub_shortest_path_edges, sub_routes)

In [ ]:
sub_shortest_path_edges

In [ ]:
sub_new_edges = add_columns_to_edges(sub_shortest_path_edges,edges)

In [ ]:
sub_new_edges

# Tram-Amsterdam

## Generate tram networks

In [ ]:
city_tram_stations, edges, nodes, city_tram_routes = prepare_tram(osm_path)

In [ ]:
edges

In [ ]:
nodes

In [ ]:
city_tram_routes

In [ ]:
len(city_tram_routes)

In [ ]:
city_tram_routes = city_tram_routes.loc[city_tram_routes.ref != 'EMA']

In [ ]:
city_tram_routes, check_name = check_tram_routes(city_tram_stations, city_tram_routes)

In [ ]:
replacement_dict = {
    ',': '',
    'Diemen': '',
    'Amsterdam': '',
    'Sloterdijk': 'Station Sloterdijk',
    'Osdorp Dijkgraafsplein': 'Dijkgraafplein',
    'Osdorp De Aker': 'Matterhorn'
}

city_tram_stations, city_tram_routes, check_name = recheck_tram_routes(replacement_dict, city_tram_stations, city_tram_routes)

In [ ]:
replacement_dict = {
    'Amstelveen Westwijk': 'Westwijk'
}

city_tram_stations, city_tram_routes, check_name = recheck_tram_routes(replacement_dict, city_tram_stations, city_tram_routes)

In [ ]:
city_tram_order_route_dict, city_tram_shortest_path_pairs, city_tram_shortest_path_edges, tram_edges = tram(city_tram_stations, edges, nodes, city_tram_routes, city='Amsterdam')

In [ ]:
tram_edges

## Add columns to nodes

In [ ]:
city_tram_order_route_dict = add_ref_to_orderroutes_or_shortestpath_dict(city_tram_order_route_dict, city_tram_routes)

In [ ]:
city_tram_order_route_dict

In [ ]:
city_tram_new_nodes = add_columns_to_nodes(city_tram_order_route_dict, nodes)

In [ ]:
city_tram_new_nodes

In [ ]:
# city_tram_new_nodes.to_excel('./city_tram_new_nodes11.xlsx',header = True, index = False)

### retrive all transfer stations on each route

In [ ]:
city_tram_transfer_stations_df = all_transfer_stations_df(city_tram_new_nodes)

In [ ]:
city_tram_transfer_stations_df

In [ ]:
len(city_tram_transfer_stations_df)

## Add columns to edges

In [ ]:
city_tram_shortest_path_edges = add_ref_to_orderroutes_or_shortestpath_dict(city_tram_shortest_path_edges, city_tram_routes)

In [ ]:
city_tram_shortest_path_edges

In [ ]:
city_tram_new_edges = add_columns_to_edges(city_tram_shortest_path_edges,edges)

In [ ]:
city_tram_new_edges

# Bus-Amsterdam

## Generate bus networks

In [ ]:
city_bus_stations, edges, nodes, city_bus_routes = prepare_bus(osm_path)

In [ ]:
edges

In [ ]:
nodes

In [ ]:
city_bus_routes

In [ ]:
len(city_bus_routes)

In [ ]:
city_bus_routes = city_bus_routes.loc[city_bus_routes.network == 'Stadsvervoer Amsterdam']
city_bus_routes = city_bus_routes.loc[city_bus_routes.service != 'night']

In [ ]:
city_bus_routes, check_name = check_bus_routes(city_bus_stations, city_bus_routes)

In [ ]:
replacement_dict = {
    'Amsterdam, ': '',
    'Amsterdam': '',
    'Weesp Station': 'Station',
    'Osdorp De Aker': 'Matterhorn',
    'Schiphol, Airport/Plaza': 'Schiphol, Airport',
    'De Boelelaan': 'De Boelelaan/De Klencke'
}
replacement_dict1 = {
    'Amsterdam, ': '',
    'Amsterdam': '',
}

city_bus_stations['name'] = city_bus_stations['name'].replace(replacement_dict1, regex=True).str.strip()

city_bus_stations, city_bus_routes, check_name = recheck_bus_routes(replacement_dict, city_bus_stations, city_bus_routes)

In [ ]:
city_bus_order_route_dict, city_bus_shortest_path_pairs, city_bus_shortest_path_edges, bus_edges = bus(city_bus_stations, edges, nodes, city_bus_routes, city='Amsterdam')

In [ ]:
city_bus_order_route_dict

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(40, 40))
gpd.GeoDataFrame(bus_edges.copy()).plot(ax=ax, color='gray', alpha=0.2)
gpd.GeoDataFrame(city_bus_shortest_path_edges['Bus 66: Amsterdam IJburg => Amsterdam Bijlmer ArenA'].copy()).plot(ax=ax, zorder=1,
                                                                     linewidth=(city_bus_shortest_path_edges['Bus 66: Amsterdam IJburg => Amsterdam Bijlmer ArenA'].count_weight) * 2,
                                                                     color='yellow')

In [ ]:
len(city_bus_order_route_dict)

In [ ]:
bus_edges

## Add columns to nodes

In [ ]:
city_bus_order_route_dict = add_ref_to_orderroutes_or_shortestpath_dict(city_bus_order_route_dict, city_bus_routes)

In [ ]:
city_bus_order_route_dict

In [ ]:
city_bus_new_nodes = add_columns_to_nodes(city_bus_order_route_dict, nodes)

In [ ]:
city_bus_new_nodes

### retrive all transfer stations on each route

In [ ]:
city_bus_transfer_stations_df = all_transfer_stations_df(city_bus_new_nodes)

In [ ]:
city_bus_transfer_stations_df

In [ ]:
len(city_bus_transfer_stations_df)

## Add columns to edges

In [ ]:
city_bus_shortest_path_edges = add_ref_to_orderroutes_or_shortestpath_dict(city_bus_shortest_path_edges, city_bus_routes)

In [ ]:
city_bus_shortest_path_edges

In [ ]:
city_bus_new_edges = add_columns_to_edges(city_bus_shortest_path_edges,edges)

In [ ]:
city_bus_new_edges

# Multi networks

### Merge three networks

In [ ]:
three_networks_edges = pd.concat([sub_new_edges,city_tram_new_edges,city_bus_new_edges])
# three_networks_nodes = pd.concat([sub_new_nodes_transfer_all,city_tram_new_nodes_transfer_all,city_bus_new_nodes_transfer_all]) 
three_networks_nodes = pd.concat([sub_new_nodes,city_tram_new_nodes,city_bus_new_nodes]) 
three_networks_routes = pd.concat([sub_routes,city_tram_routes,city_bus_routes])

In [ ]:
three_networks_edges

In [ ]:
three_networks_nodes

In [ ]:
type(three_networks_nodes.geometry[0])

In [ ]:
three_networks_routes

In [ ]:
merged_order_route_dict = {key: value for d in [sub_order_route_dict, city_tram_order_route_dict, city_bus_order_route_dict] for key, value in d.items()}

In [ ]:
merged_order_route_dict

# Adding ref_transfer_all column and get three_network_nodes_transfer_all

In [ ]:
# convert coordinates in station_df from degree to meter
def convert_deg_to_meter(new_nodes_df):
    # Convert the 'geometry' column to a Point object
    new_nodes_df['geometry'] = new_nodes_df.apply(lambda row: Point(row['geo_x'], row['geo_y']), axis=1)
    
    # Define the coordinate reference systems (CRS)
    crs_deg = 'EPSG:4326'  # WGS 84, the standard coordinate system for GPS
    crs_meter = 'EPSG:3857'  # World Mercator, commonly used for distance calculations

    # Create a transformer to convert from degrees to meters
    transformer = Transformer.from_crs(crs_deg, crs_meter, always_xy=True)

    # Apply the transformer to each Point in the 'geometry' column
    new_nodes_df['geometry'] = new_nodes_df['geometry'].apply(lambda point: Point(transformer.transform(point.x, point.y)))
    
    return new_nodes_df

In [ ]:
def new_nodes_buffer(new_nodes_df,buffer_radius):
    new_nodes_df = convert_deg_to_meter(new_nodes_df)
    node_buffers = []

    for index, row in new_nodes_df.iterrows():
        point = row['geometry']
        buffer = point.buffer(buffer_radius)
        node_buffers.append(buffer)

    buffered_gdf = gpd.GeoDataFrame(geometry=node_buffers)
    buffered_df = pd.DataFrame(buffered_gdf).rename(columns={'geometry': 'buffer_geometry'})
    new_nodes_buffer_geometry = pd.concat([new_nodes_df,buffered_df],axis=1)
    return new_nodes_buffer_geometry

In [ ]:
def add_footnote_to_new_nodes(new_nodes_df):
    new_nodes_df = convert_deg_to_meter(new_nodes_df)
    # create a exactly same dataframe with new_nodes_df but name columns by different transportation routes
    rename_cols = {
    "geometry": "geometry"+"_"+new_nodes_df.loc[0,'route'],
    "degree": "degree"+"_"+new_nodes_df.loc[0,'route'],
    "id": "id"+"_"+new_nodes_df.loc[0,'route'],
    "name": "name"+"_"+new_nodes_df.loc[0,'route'],
    "geo_x": "geo_x"+"_"+new_nodes_df.loc[0,'route'],
    "geo_y": "geo_y"+"_"+new_nodes_df.loc[0,'route'],
    "coordinate_value": "coordinate_value"+"_"+new_nodes_df.loc[0,'route'],
    "route_name_list": "route_name_list"+"_"+new_nodes_df.loc[0,'route'],
    "route": "route"+"_"+new_nodes_df.loc[0,'route'],
    "ref": "ref"+"_"+new_nodes_df.loc[0,'route'],
    "transfer": "transfer"+"_"+new_nodes_df.loc[0,'route']
    }
    new_nodes_ftn = new_nodes_df.rename(columns=rename_cols)
    return new_nodes_ftn

In [ ]:
def transfer_A_to_B(A_new_nodes_buffer_geometry,B_new_nodes_ftn):
    transfer_stations_A_to_B = pd.DataFrame()
    
    for index, row in A_new_nodes_buffer_geometry.iterrows():
        each_transfer_stations_df = B_new_nodes_ftn[row['buffer_geometry'].intersects(B_new_nodes_ftn.iloc[:, 0])]
        
        if not each_transfer_stations_df.empty:
            replicated_row = pd.concat([row] * len(each_transfer_stations_df), axis=1).transpose()
            replicated_row.reset_index(drop=True, inplace=True)  # Reset the index
            each_transfer_stations_df.reset_index(drop=True, inplace=True)  # Reset the index
            co_df = pd.concat([replicated_row,each_transfer_stations_df], axis=1)
            transfer_stations_A_to_B = pd.concat([transfer_stations_A_to_B,co_df])
            
       

    return transfer_stations_A_to_B

In [ ]:
def process_ref_transfer_all(cell_value):
    if pd.notna(cell_value):  # 检查是否是 NaN 值
        return set(filter(lambda x: x != 'nan', map(str.strip, cell_value.split(','))))
    else:
        return set()

In [ ]:
def add_ref_transfer_all_to_new_nodes(transfer_stations_A_to_B, transfer_stations_A_to_C, A_new_nodes):

    selected_columns_to_B = transfer_stations_A_to_B.iloc[:, [0, 3, 9,21]]

    unique_rows_to_B = selected_columns_to_B.drop_duplicates(subset='geometry')

    unique_rows_to_B['ref_1'] = unique_rows_to_B.apply(lambda row: f"{row.iloc[2]}, {row.iloc[3]}", axis=1)


    transfer_ref_to_B = unique_rows_to_B[['name', 'ref_1']].drop_duplicates(subset='name')

    A_new_nodes_to_B = pd.merge(A_new_nodes,transfer_ref_to_B,on = 'name', how = 'left')

    
    selected_columns_to_C = transfer_stations_A_to_C.iloc[:, [0, 3, 9,21]]

    unique_rows_to_C = selected_columns_to_C.drop_duplicates(subset='geometry')

    unique_rows_to_C['ref_2'] = unique_rows_to_C.apply(lambda row: f"{row.iloc[2]}, {row.iloc[3]}", axis=1)


    transfer_ref_to_C = unique_rows_to_C[['name', 'ref_2']].drop_duplicates(subset='name')

    A_new_nodes_to_B_C = pd.merge(A_new_nodes_to_B,transfer_ref_to_C,on = 'name', how = 'left')
    

    A_new_nodes_to_B_C['ref_transfer_all'] = A_new_nodes_to_B_C.apply(lambda row: f"{row['ref']}, {row['ref_1']}, {row['ref_2']}", axis=1)
    
    A_new_nodes_to_B_C['ref_transfer_all'] = A_new_nodes_to_B_C['ref_transfer_all'].apply(process_ref_transfer_all)

    A_new_nodes_transfer_all = A_new_nodes_to_B_C
    

    return A_new_nodes_transfer_all

In [ ]:
buffer_radius = 200

In [ ]:
sub_new_nodes_buffer_geometry = new_nodes_buffer(sub_new_nodes,buffer_radius)

city_tram_new_nodes_ftn = add_footnote_to_new_nodes(city_tram_new_nodes)
transfer_stations_sub_to_tram = transfer_A_to_B(sub_new_nodes_buffer_geometry,city_tram_new_nodes_ftn)

city_bus_new_nodes_ftn = add_footnote_to_new_nodes(city_bus_new_nodes)
transfer_stations_sub_to_bus = transfer_A_to_B(sub_new_nodes_buffer_geometry,city_bus_new_nodes_ftn)

sub_new_nodes_transfer_all = add_ref_transfer_all_to_new_nodes(transfer_stations_sub_to_tram, transfer_stations_sub_to_bus, sub_new_nodes)
sub_new_nodes_transfer_all

In [ ]:
city_tram_new_nodes_buffer_geometry = new_nodes_buffer(city_tram_new_nodes,buffer_radius)

sub_new_nodes_ftn = add_footnote_to_new_nodes(sub_new_nodes)
transfer_stations_tram_to_sub = transfer_A_to_B(city_tram_new_nodes_buffer_geometry,sub_new_nodes_ftn)

city_bus_new_nodes_ftn = add_footnote_to_new_nodes(city_bus_new_nodes)
transfer_stations_tram_to_bus = transfer_A_to_B(city_tram_new_nodes_buffer_geometry,city_bus_new_nodes_ftn)

city_tram_new_nodes_transfer_all = add_ref_transfer_all_to_new_nodes(transfer_stations_tram_to_sub, transfer_stations_tram_to_bus, city_tram_new_nodes)
city_tram_new_nodes_transfer_all

In [ ]:
city_bus_new_nodes_buffer_geometry = new_nodes_buffer(city_bus_new_nodes,buffer_radius)

sub_new_nodes_ftn = add_footnote_to_new_nodes(sub_new_nodes)
transfer_stations_bus_to_sub = transfer_A_to_B(city_bus_new_nodes_buffer_geometry,sub_new_nodes_ftn)

city_tram_new_nodes_ftn = add_footnote_to_new_nodes(city_tram_new_nodes)
transfer_stations_bus_to_tram = transfer_A_to_B(city_bus_new_nodes_buffer_geometry,city_tram_new_nodes_ftn)

city_bus_new_nodes_transfer_all = add_ref_transfer_all_to_new_nodes(transfer_stations_bus_to_sub, transfer_stations_bus_to_tram, city_bus_new_nodes)
city_bus_new_nodes_transfer_all

In [ ]:
three_networks_nodes_transfer_all = pd.concat([sub_new_nodes_transfer_all,city_tram_new_nodes_transfer_all,city_bus_new_nodes_transfer_all]).reset_index(drop=True)

In [ ]:
three_networks_nodes_transfer_all

# Create original points dataframe and OD pair list

In [ ]:
sub_order_route_dict

In [ ]:
origin_df = pd.DataFrame()
each_o_point = pd.DataFrame()
for route_name, station_df in sub_order_route_dict.items():
    each_o_point = station_df.loc[[0]]
    origin_df = pd.concat([origin_df,each_o_point])
origin_df.drop_duplicates(subset='geometry', inplace=True)
selected_columns = ['geometry','geo_x','geo_y','name']
origin_df = origin_df[selected_columns].reset_index(drop=True)
origin_df = convert_deg_to_meter(origin_df)
origin_df['o_geo_x'] = origin_df['geometry'].apply(lambda point: point.x)
origin_df['o_geo_y'] = origin_df['geometry'].apply(lambda point: point.y)
origin_df

In [ ]:
# rename_cols = {
#     "geometry": "o_geometry",
#     "geo_x": "o_geo_x",
#     "geo_y": "o_geo_y"
#     }
# origin_df = origin_df.rename(columns=rename_cols)
# origin_df

In [ ]:
destination_df1 = pd.DataFrame()
each_d_point = pd.DataFrame()
for route_name, station_df in city_tram_order_route_dict.items():
    each_d_point = station_df.loc[[0]]
    destination_df1 = pd.concat([destination_df1,each_d_point])
destination_df1.drop_duplicates(subset='geometry', inplace=True)
selected_columns = ['geometry', 'geo_x', 'geo_y']
destination_df1 = destination_df1[selected_columns].reset_index(drop=True)
destination_df1

In [ ]:
destination_df2 = pd.DataFrame()
each_d_point = pd.DataFrame()
for route_name, station_df in city_bus_order_route_dict.items():
    each_d_point = station_df.loc[[0]]
    destination_df2 = pd.concat([destination_df2,each_d_point])
destination_df2.drop_duplicates(subset='geometry', inplace=True)
selected_columns = ['geometry', 'geo_x', 'geo_y']
destination_df2 = destination_df2[selected_columns].reset_index(drop=True)
destination_df2

In [ ]:
destination_df = pd.concat([destination_df1,destination_df2])
destination_df.drop_duplicates(subset='geometry', inplace=True)
destination_df = destination_df.reset_index(drop=True)
destination_df = convert_deg_to_meter(destination_df)
destination_df['d_geo_x'] = destination_df['geometry'].apply(lambda point: point.x)
destination_df['d_geo_y'] = destination_df['geometry'].apply(lambda point: point.y)
destination_df

In [ ]:
# rename_cols = {
#     "geometry": "d_geometry",
#     "geo_x": "d_geo_x",
#     "geo_y": "d_geo_y"
#     }
# destination_df = destination_df.rename(columns=rename_cols)

# destination_df

In [ ]:
OD_list = []
for index, row_origin in origin_df.iterrows():
    o_geo_x = row_origin['o_geo_x']
    o_geo_y = row_origin['o_geo_y']
       
    for index, row_destination in destination_df.iterrows():
        d_geo_x = row_destination['d_geo_x']
        d_geo_y = row_destination['d_geo_y']
        
        OD_dict = {'s_coordinates_x': o_geo_x, 's_coordinates_y': o_geo_y, 'e_coordinates_x': d_geo_x, 'e_coordinates_y': d_geo_y}
        OD_list.append(OD_dict)        

In [ ]:
OD_list

In [ ]:
len(OD_list)

In [ ]:
OD_list[50]

# Situation1-1: Test and revise transfer functions with OD_list and three_network_nodes_transfer_all----- on one same route

In [ ]:
start_node, end_node = s_e_node_df(OD_list[50], three_networks_nodes_transfer_all)

In [ ]:
start_node

In [ ]:
end_node

### Find the route which the s_e nodes are on----compare with new_edges instead of routes dataframe, should compare with routes dataframe

In [ ]:
def s_e_on_route_ref_transfer_all(start_node, end_node):
    
    start_node_ref_transfer_all = start_node.ref_transfer_all.to_list()
    s_on_route_ref_transfer_all = pd.DataFrame(list(start_node_ref_transfer_all[0]), columns=['ref'])

    end_node_ref_transfer_all = end_node.ref_transfer_all.to_list()
    e_on_route_ref_transfer_all = pd.DataFrame(list(end_node_ref_transfer_all[0]), columns=['ref'])

    return s_on_route_ref_transfer_all, e_on_route_ref_transfer_all

In [ ]:
s_on_route_ref,e_on_route_ref = s_e_on_route_ref_transfer_all(start_node, end_node)

In [ ]:
s_on_route_ref

In [ ]:
e_on_route_ref

### Make judgement of s_e_on_route_ref

In [ ]:
s_on_route_ref, e_on_route_ref = judge_on_route(s_on_route_ref, e_on_route_ref)

### Find those same routes

In [ ]:
s_e_same_routes_df = s_e_same_routes(s_on_route_ref,e_on_route_ref)
s_e_same_routes_df

### Get the same route information--- A route gdf

In [ ]:
s_e_on_possible_route_df = pd.merge(s_e_same_routes_df, three_networks_routes, on='ref', how='left')
s_e_on_possible_route_df

In [ ]:
s_e_on_route_possible_gdf = gpd.GeoDataFrame(s_e_on_possible_route_df.copy()).reset_index(drop=True)

start_node_gdf = gpd.GeoDataFrame(start_node.copy())
start_node_gdf.crs = "EPSG:3857"
start_node_gdf = start_node_gdf.to_crs("EPSG:4326")
end_node_gdf = gpd.GeoDataFrame(end_node.copy())
end_node_gdf.crs = "EPSG:3857"
end_node_gdf = end_node_gdf.to_crs("EPSG:4326")

In [ ]:
s_e_on_route_possible_gdf

In [ ]:
start_node_gdf.iloc[0]['geometry']

In [ ]:
s_e_on_route_possible_gdf.geometry.buffer(0.000001).intersects(start_node_gdf.iloc[0]['geometry'])

In [ ]:
s_e_on_same_route_gdf = s_e_on_route_possible_gdf[s_e_on_route_possible_gdf.geometry.buffer(0.000001).intersects(start_node_gdf.iloc[0]['geometry'])]
s_e_on_same_route_gdf

In [ ]:
s_e_on_route_possible_gdf.geometry.buffer(0.000001).intersects(end_node_gdf.iloc[0]['geometry'])

In [ ]:
def s_e_same_routes_transfer_all(s_e_same_routes_df,three_networks_routes,start_node):
    s_e_on_possible_route_df = pd.merge(s_e_same_routes_df, three_networks_routes, on='ref', how='left')
    s_e_on_route_possible_gdf = gpd.GeoDataFrame(s_e_on_possible_route_df.copy())

    start_node_gdf = gpd.GeoDataFrame(start_node.copy())
    start_node_gdf.crs = "EPSG:3857"
    start_node_gdf = start_node_gdf.to_crs("EPSG:4326")

    s_e_on_same_route_gdf = s_e_on_route_possible_gdf[s_e_on_route_possible_gdf.geometry.buffer(0.000001).intersects(start_node_gdf.iloc[0]['geometry'])]
    return start_node_gdf,s_e_on_same_route_gdf

In [ ]:
start_node_gdf,s_e_on_same_route_gdf = s_e_same_routes_transfer_all(s_e_same_routes_df,three_networks_routes,start_node)

In [ ]:
s_e_on_same_route_gdf

In [ ]:
def s_e_same_routes_transfer_all(s_e_same_routes_df,three_networks_routes,start_node):
    s_e_on_same_route_gdf = pd.DataFrame(three_networks_routes[three_networks_routes.name == start_node_gdf.iloc[0]['route_name_list']]).reset_index(drop=True)
    return s_e_on_same_route_gdf

In [ ]:
s_e_on_same_route_gdf = s_e_same_routes_transfer_all(s_e_same_routes_df,three_networks_routes,start_node)

In [ ]:
s_e_on_same_route_gdf

### Get all stations between s_e nodes

In [ ]:
def all_btw_stations_on_same_route_gdf(merged_order_route_dict, s_e_on_same_route_gdf, start_node_gdf):
    all_stations_on_same_route = merged_order_route_dict[s_e_on_same_route_gdf.name[0]]
   
    all_stations_on_same_route_gdf = gpd.GeoDataFrame(all_stations_on_same_route.copy())
    all_stations_on_same_route_gdf.crs = "EPSG:3857"
    end_node_gdf = gpd.GeoDataFrame(end_node.copy())
    end_node_gdf.crs = "EPSG:3857"
    end_node_gdf = end_node_gdf.to_crs("EPSG:4326")

    start_station_index = shapely.distance(start_node_gdf.geometry.values,all_stations_on_same_route_gdf.geometry.values).argmin()
    end_station_index = shapely.distance(end_node_gdf.geometry.values,all_stations_on_same_route_gdf.geometry.values).argmin()
    all_btw_stations_on_same_route_gdf = all_stations_on_same_route_gdf.iloc[start_station_index:end_station_index+1]

    return all_btw_stations_on_same_route_gdf

In [ ]:
all_btw_stations_on_same_route_gdf = all_btw_stations_on_same_route_gdf(merged_order_route_dict, s_e_on_same_route_gdf, start_node_gdf)
all_btw_stations_on_same_route_gdf

In [ ]:
# all_stations_on_same_route_gdf.iloc[shapely.distance(end_node_gdf.geometry.values,all_stations_on_same_route_gdf.geometry.values).argmin()]

### Get all nodes and id_pairs of nodes between s_e nodes

In [ ]:
def btw_all_ids_pairs_transfer_all(all_btw_stations_on_same_route_gdf, three_networks_nodes_transfer_all):
    s_e_between_nodes = three_networks_nodes_transfer_all.merge(all_btw_stations_on_same_route_gdf, on = 'coordinate_value', suffixes=('', '_drop'))

    btw_all_id_pairs_list = []
    s_e_btw_nodes_transfer_all = three_networks_nodes_transfer_all.merge(all_btw_stations_on_same_route_gdf, on = 'coordinate_value', suffixes=('', '_drop'))

   

    id_pairs_way1 = pd.DataFrame(columns=['s_id', 'e_id'])

  
    for i in range(len(s_e_btw_nodes_transfer_all)-1):
           
        id_pairs_way1.loc[i, 's_id'] = s_e_btw_nodes_transfer_all.loc[i, 'id']
        id_pairs_way1.loc[i, 'e_id'] = s_e_btw_nodes_transfer_all.loc[i+1, 'id']
        

    btw_all_id_pairs_list.append(id_pairs_way1)

    return btw_all_id_pairs_list

In [ ]:
btw_all_id_pairs_list = btw_all_ids_pairs_transfer_all(all_btw_stations_on_same_route_gdf,three_networks_nodes_transfer_all)
btw_all_id_pairs_list

### Calculate shortest path and plot

In [ ]:
btw_all_id_pairs_list_unique = drop_df_in_list(btw_all_id_pairs_list)

In [ ]:
btw_all_id_pairs_list_unique

In [ ]:
def judge_merged_df_for_G(s_e_on_same_route_gdf,sub_edges,tram_edges,bus_edges):
    
    if s_e_on_same_route_gdf.route[0] == 'subway':
        merged_df = sub_edges['geometry']
        return merged_df
    elif s_e_on_same_route_gdf.route[0] == 'tram':
        merged_df = tram_edges['geometry']        
        return merged_df
    elif s_e_on_same_route_gdf.route[0] == 'bus':
        merged_df = bus_edges['geometry']
        return merged_df
    else:
        print('There maybe someting wrong in s_e_on_same_route_gdf')


def edges_df_G_shortest(s_e_on_same_route_gdf,sub_edges,tram_edges,bus_edges,three_networks_edges):
    merged_df = judge_merged_df_for_G(s_e_on_same_route_gdf,sub_edges,tram_edges,bus_edges)
    
    edges_df_G_shortest = pd.merge(merged_df,three_networks_edges, on = 'geometry', how = 'inner')
    
    columns_to_delete = ['count_weight', 'route_name_list',	'route', 'ref']
    edges_df_G_shortest = edges_df_G_shortest.drop(columns_to_delete, axis=1)
    
    return edges_df_G_shortest

In [ ]:
edges_df_G_shortest = edges_df_G_shortest(s_e_on_same_route_gdf,sub_edges,tram_edges,bus_edges,three_networks_edges)

In [ ]:
G = create_ground_graph(edges_df_G_shortest, three_networks_nodes)

In [ ]:
shortest_path_pairs_way = all_shortest_paths(G, btw_all_id_pairs_list_unique[0], edges_df_G_shortest)

In [ ]:
shortest_path_pairs_way

In [ ]:
duplicate_row_count_way, shortest_path_edges_way, edges_way = edges_with_count_weight(shortest_path_pairs_way, edges_df_G_shortest)

In [ ]:
shortest_path_edges_way

In [ ]:
shortest_path_edges_way.time.sum()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(30, 20))

gpd.GeoDataFrame(three_networks_edges.copy()).plot(ax=ax, color='gray', alpha=0.2)

gpd.GeoDataFrame(shortest_path_edges_way.copy()).plot(ax=ax, zorder=1, linewidth=(shortest_path_edges_way.count_weight), color='orange')

In [ ]:
sub_edges

In [ ]:
G = create_ground_graph(sub_edges, three_networks_nodes)

In [ ]:
shortest_path_pairs_way = all_shortest_paths(G, btw_all_id_pairs_list_unique[0], sub_edges)

In [ ]:
shortest_path_pairs_way

In [ ]:
duplicate_row_count_way, shortest_path_edges_way, edges_way = edges_with_count_weight(shortest_path_pairs_way, sub_edges)

In [ ]:
shortest_path_edges_way

In [ ]:
shortest_path_edges_way.time.sum()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(30, 20))

gpd.GeoDataFrame(city_bus_new_edges.copy()).plot(ax=ax, color='gray', alpha=0.2)

gpd.GeoDataFrame(shortest_path_edges_way.copy()).plot(ax=ax, zorder=1, linewidth=(shortest_path_edges_way.count_weight), color='orange')

In [ ]:
sub_edges_geometry = sub_edges['geometry']
sub_edges_geometry

In [ ]:
all_btw_stations_on_edges = pd.merge(sub_edges_geometry,three_networks_edges, on = 'geometry', how = 'inner')
all_btw_stations_on_edges

In [ ]:
columns_to_delete = ['count_weight', 'route_name_list',	'route']
all_btw_stations_on_edges_for_shortest = all_btw_stations_on_edges.drop(columns_to_delete, axis=1)
all_btw_stations_on_edges_for_shortest

In [ ]:
G = create_ground_graph(all_btw_stations_on_edges_for_shortest, three_networks_nodes)

In [ ]:
shortest_path_pairs_way = all_shortest_paths(G, btw_all_id_pairs_list_unique[0], all_btw_stations_on_edges_for_shortest)

In [ ]:
shortest_path_pairs_way

In [ ]:
duplicate_row_count_way, shortest_path_edges_way, edges_way = edges_with_count_weight(shortest_path_pairs_way, all_btw_stations_on_edges_for_shortest)

In [ ]:
shortest_path_edges_way

In [ ]:
shortest_path_edges_way.time.sum()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(30, 20))

gpd.GeoDataFrame(city_bus_new_edges.copy()).plot(ax=ax, color='gray', alpha=0.2)

gpd.GeoDataFrame(shortest_path_edges_way.copy()).plot(ax=ax, zorder=1, linewidth=(shortest_path_edges_way.count_weight), color='orange')

# Situation1-2: Test and revise transfer functions with OD_list and three_network_nodes_transfer_all----- on more than one same routes

In [ ]:
start_node, end_node = s_e_node_df(OD_list[55], three_networks_nodes_transfer_all)

In [ ]:
start_node

In [ ]:
end_node

### Find the route which the s_e nodes are on----compare with new_edges instead of routes dataframe, should compare with routes dataframe

In [ ]:
def s_e_on_route_ref_transfer_all(start_node, end_node):
    
    start_node_ref_transfer_all = start_node.ref_transfer_all.to_list()
    s_on_route_ref_transfer_all = pd.DataFrame(list(start_node_ref_transfer_all[0]), columns=['ref'])

    end_node_ref_transfer_all = end_node.ref_transfer_all.to_list()
    e_on_route_ref_transfer_all = pd.DataFrame(list(end_node_ref_transfer_all[0]), columns=['ref'])

    return s_on_route_ref_transfer_all, e_on_route_ref_transfer_all

In [ ]:
s_on_route_ref,e_on_route_ref = s_e_on_route_ref_transfer_all(start_node, end_node)

In [ ]:
s_on_route_ref

In [ ]:
e_on_route_ref

### Make judgement of s_e_on_route_ref

In [ ]:
s_on_route_ref, e_on_route_ref = judge_on_route(s_on_route_ref, e_on_route_ref)

### Find those same routes

In [ ]:
s_e_same_routes_df = s_e_same_routes(s_on_route_ref,e_on_route_ref)
s_e_same_routes_df

### Get all stations between s_e nodes and transfer nodes

In [ ]:
def s_e_same_routes_transfer_all(s_e_same_routes_df,three_networks_routes,start_node):
    s_e_on_possible_route_df = pd.merge(s_e_same_routes_df, three_networks_routes, on='ref', how='left')
    s_e_on_route_possible_gdf = gpd.GeoDataFrame(s_e_on_possible_route_df.copy())

    start_node_gdf = gpd.GeoDataFrame(start_node.copy())
    start_node_gdf.crs = "EPSG:3857"
    start_node_gdf = start_node_gdf.to_crs("EPSG:4326")
    
    s_e_on_route_possible_gdf = s_e_on_route_possible_gdf.reset_index(drop=True)
    start_node_gdf = start_node_gdf.reset_index(drop=True)
    
    s_e_on_same_route_gdf = s_e_on_route_possible_gdf[s_e_on_route_possible_gdf.geometry.buffer(0.000001).intersects(start_node_gdf.geometry)]
    return s_e_on_same_route_gdf

In [ ]:
s_e_on_same_route_gdf = s_e_same_routes_transfer_all(s_e_same_routes_df,three_networks_routes,start_node)
s_e_on_same_route_gdf

In [ ]:
len(s_e_same_routes_df)

In [ ]:
s_e_on_possible_route_df = pd.merge(s_e_same_routes_df, three_networks_routes, on='ref', how='left')
s_e_on_possible_route_df

In [ ]:
s_e_on_route_possible_gdf = gpd.GeoDataFrame(s_e_on_possible_route_df.copy())

start_node_gdf = gpd.GeoDataFrame(start_node.copy())
start_node_gdf.crs = "EPSG:3857"
start_node_gdf = start_node_gdf.to_crs("EPSG:4326")

In [ ]:
start_node_gdf

In [ ]:
s_e_on_route_possible_gdf.geometry[1]

In [ ]:
s_e_on_route_possible_gdf.geometry[2].buffer(0.000001).intersects(start_node_gdf.geometry)

In [ ]:
s_e_on_same_route_gdf = s_e_on_route_possible_gdf[s_e_on_route_possible_gdf.geometry.buffer(0.000001).intersects(start_node_gdf.geometry)]

In [ ]:
s_e_on_same_route_gdf

In [ ]:
# Resetting the index of GeoDataFrames
s_e_on_route_possible_gdf = s_e_on_route_possible_gdf.reset_index(drop=True)
start_node_gdf = start_node_gdf.reset_index(drop=True)

# Performing the intersection operation
s_e_on_same_route_gdf = s_e_on_route_possible_gdf[s_e_on_route_possible_gdf.geometry.buffer(0.000001).intersects(start_node_gdf.geometry)]

In [ ]:
s_e_on_same_route_gdf

In [ ]:
s_e_on_same_route_gdf = s_e_on_route_possible_gdf[s_e_on_route_possible_gdf.geometry[1].buffer(0.000001).intersects(start_node_gdf.geometry)]
s_e_on_same_route_gdf

In [ ]:
merged_order_route_dict[s_e_on_same_route_gdf.name[0]]

# Situation2: Test and revise transfer functions with OD_list and three_network_nodes_transfer_all

In [ ]:
start_node, end_node = s_e_node_df(OD_list[7], three_networks_nodes_transfer_all)

In [ ]:
start_node

In [ ]:
end_node

### Find the route which the s_e nodes are on----compare with new_edges instead of routes dataframe, should compare with routes dataframe

In [ ]:
def s_e_on_route_ref_transfer_all(start_node, end_node):
    
    start_node_ref_transfer_all = start_node.ref_transfer_all.to_list()
    s_on_route_ref_transfer_all = pd.DataFrame(list(start_node_ref_transfer_all[0]), columns=['ref'])

    end_node_ref_transfer_all = end_node.ref_transfer_all.to_list()
    e_on_route_ref_transfer_all = pd.DataFrame(list(end_node_ref_transfer_all[0]), columns=['ref'])

    return s_on_route_ref_transfer_all, e_on_route_ref_transfer_all

In [ ]:
s_on_route_ref,e_on_route_ref = s_e_on_route_ref_transfer_all(start_node, end_node)

In [ ]:
s_on_route_ref

In [ ]:
e_on_route_ref

### Make judgement of s_e_on_route_ref

In [ ]:
s_on_route_ref, e_on_route_ref = judge_on_route(s_on_route_ref, e_on_route_ref)

### Find those same routes

In [ ]:
s_e_same_routes_df = s_e_same_routes(s_on_route_ref,e_on_route_ref)
s_e_same_routes_df

### Find transfer stations----transfer_station_one_mode

In [ ]:
t_station_node = transfer_station_one_mode(city_bus_routes, start_node, end_node, city_bus_new_nodes, city_bus_order_route_dict)

In [ ]:
t_station_node

### Get all stations between s_e nodes and transfer nodes

In [ ]:
s_on_route_ref,t_on_route_ref = s_e_on_route_ref(start_node, t_station_node)

In [ ]:
s_t_same_routes_df = s_e_same_routes(s_on_route_ref,t_on_route_ref)
s_t_same_routes_df

In [ ]:
all_stations_on_matched_routes_dfs_s_t = all_stations_on_matched_routes(s_t_same_routes_df,city_bus_routes,start_node,t_station_node,city_bus_order_route_dict)
all_stations_on_matched_routes_dfs_s_t

In [ ]:
btw_stations_each_way_list_s_t = btw_stations_each_way_list(start_node,t_station_node,all_stations_on_matched_routes_dfs_s_t)
btw_stations_each_way_list_s_t

In [ ]:
t_on_route_ref,e_on_route_ref = s_e_on_route_ref( t_station_node,end_node)

In [ ]:
t_e_same_routes_df = s_e_same_routes(t_on_route_ref,e_on_route_ref)
t_e_same_routes_df

In [ ]:
all_stations_on_matched_routes_dfs_t_e = all_stations_on_matched_routes(t_e_same_routes_df,city_bus_routes,t_station_node,end_node,city_bus_order_route_dict)
all_stations_on_matched_routes_dfs_t_e

In [ ]:
btw_stations_each_way_list_t_e = btw_stations_each_way_list(t_station_node,end_node,all_stations_on_matched_routes_dfs_t_e)
btw_stations_each_way_list_t_e

### Get all nodes and id_pairs of nodes between s_e nodes

In [ ]:
btw_all_id_pairs_list_s_t = btw_all_ids_pairs(btw_stations_each_way_list_s_t,city_bus_new_nodes)
btw_all_id_pairs_list_s_t

In [ ]:
btw_all_id_pairs_list_t_e = btw_all_ids_pairs(btw_stations_each_way_list_t_e,city_bus_new_nodes)
btw_all_id_pairs_list_t_e

### Calculate shortest path and plot

In [ ]:
btw_all_id_pairs_list_unique_s_t = drop_df_in_list(btw_all_id_pairs_list_s_t)

In [ ]:
btw_all_id_pairs_list_unique_t_e = drop_df_in_list(btw_all_id_pairs_list_t_e)

In [ ]:
G = create_ground_graph(edges, nodes)

In [ ]:
shortest_path_pairs_way_s_t = all_shortest_paths(G, btw_all_id_pairs_list_unique_s_t[0], edges)

In [ ]:
shortest_path_pairs_way_s_t

In [ ]:
duplicate_row_count_way_s_t, shortest_path_edges_way_s_t, edges_way_s_t = edges_with_count_weight(shortest_path_pairs_way_s_t, edges)

In [ ]:
shortest_path_edges_way_s_t

In [ ]:
shortest_path_pairs_way_t_e = all_shortest_paths(G, btw_all_id_pairs_list_unique_t_e[0], edges)

In [ ]:
shortest_path_pairs_way_t_e

In [ ]:
duplicate_row_count_way_t_e, shortest_path_edges_way_t_e, edges_way_t_e = edges_with_count_weight(shortest_path_pairs_way_t_e, edges)

In [ ]:
shortest_path_edges_way_t_e

In [ ]:
shortest_path_edges_way = pd.concat([shortest_path_edges_way_s_t,shortest_path_edges_way_t_e])
shortest_path_edges_way

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(30, 20))

gpd.GeoDataFrame(city_bus_new_edges.copy()).plot(ax=ax, color='gray', alpha=0.2)

gpd.GeoDataFrame(shortest_path_edges_way.copy()).plot(ax=ax, zorder=1, linewidth=(shortest_path_edges_way.count_weight), color='orange')